In [13]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class DebrisDataset(Dataset):
    def __init__(self, main_dir, transform=None):
        self.main_dir = main_dir
        self.transform = transform
        self.samples = []

        # List all model directories in main_dir
        model_dirs = [d for d in os.listdir(main_dir) if os.path.isdir(os.path.join(main_dir, d))]

        for model_dir in model_dirs:
            model_path = os.path.join(main_dir, model_dir, "04_FinalProcessedData")
            elevation_path = os.path.join(model_path, "elevation", f"{model_dir}_elevation.npy")
            thickness_path = os.path.join(model_path, "thickness")
            velocity_path = os.path.join(model_path, "velocity")

            # Check if elevation file exists
            if os.path.isfile(elevation_path):
                # List and sort thickness and velocity files
                thickness_files = sorted([f for f in os.listdir(thickness_path) if f.startswith(f"{model_dir}_thickness")])
                velocity_files = sorted([f for f in os.listdir(velocity_path) if f.startswith(f"{model_dir}_velocity")])

                # Assuming there is a one-to-one correspondence between thickness and velocity files
                for thickness_file, velocity_file in zip(thickness_files, velocity_files):
                    thickness_full_path = os.path.join(thickness_path, thickness_file)
                    velocity_full_path = os.path.join(velocity_path, velocity_file)

                    # Add a tuple of paths to the samples list
                    self.samples.append((elevation_path, thickness_full_path, velocity_full_path))

        # Check and print the number of samples found after trying to populate self.samples
        if not self.samples:
            print("No samples found. Check the dataset directory structure and paths.")
        else:
            print(f"Found {len(self.samples)} samples in the dataset.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        elevation_path, thickness_path, velocity_path = self.samples[idx]

        elevation = np.load(elevation_path)
        thickness = np.load(thickness_path)
        velocity = np.load(velocity_path)

        # Stack the arrays to create a 3-band image (channels first)
        image = np.stack([elevation, thickness, velocity])

        if self.transform:
            image = self.transform(image)

        # For simplicity, let's predict the next state as the velocity
        # You may need to adjust this depending on your exact requirement
        target = velocity

        return torch.from_numpy(image).float(), torch.from_numpy(target).float()

# Define any transformations if needed - for example normalization
transform = transforms.Compose([
    # Add any required transforms here
])

# Create the dataset
main_dir = r'/home/tom/repos/dyna-landslide-surrogate/data_small'
dataset = DebrisDataset(main_dir, transform=transform)

# Split the dataset into train and validation sets if needed
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

Found 1083 samples in the dataset.


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet(nn.Module):
    """A U-Net architecture for semantic segmentation."""

    def __init__(self, in_channels, out_channels):
        """Initializes the UNet with the given number of input and output channels."""
        super(UNet, self).__init__()

        # Encoder
        self.enc1 = self.encoder_block(in_channels, 8)
        self.enc2 = self.encoder_block(8, 16)
        self.enc3 = self.encoder_block(16, 32)
        self.enc4 = self.encoder_block(32, 64)

        # Bottleneck
        self.bottleneck = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU()
        )

        # Decoder
        self.dec1 = self.decoder_block(128 + 64, 64)  # Adjusted for concatenated channels
        self.dec2 = self.decoder_block(64 + 32, 32)   # Adjusted for concatenated channels
        self.dec3 = self.decoder_block(32 + 16, 16)   # Adjusted for concatenated channels
        self.dec4 = self.decoder_block(16 + 8, 8)     # Adjusted for concatenated channels

        # Final output
        self.out_conv = nn.Conv2d(8, out_channels, kernel_size=1)

    def encoder_block(self, in_channels, out_channels):
        """Defines an encoder block with Convolution, ReLU activation, and MaxPooling."""
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def decoder_block(self, in_channels, out_channels):
        """Defines a decoder block with Convolution, ReLU activation, and Upsampling."""
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        )

    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)
        #print(f'e1 shape: {e1.shape}')
        e2 = self.enc2(e1)
        #print(f'e2 shape: {e2.shape}')
        e3 = self.enc3(e2)
        #print(f'e3 shape: {e3.shape}')
        e4 = self.enc4(e3)
        #print(f'e4 shape: {e4.shape}')

        # Bottleneck
        b = self.bottleneck(e4)
        #print(f'b shape: {b.shape}')

        # Decoder with skip connections and cropping
        d1 = self.dec1(torch.cat((self.crop(e4, b), b), dim=1))
       # print(f'd1 shape: {d1.shape}')
        d2 = self.dec2(torch.cat((self.crop(e3, d1), d1), dim=1))
        #print(f'd2 shape: {d2.shape}')
        d3 = self.dec3(torch.cat((self.crop(e2, d2), d2), dim=1))
        #print(f'd3 shape: {d3.shape}')
        d4 = self.dec4(torch.cat((self.crop(e1, d3), d3), dim=1))  # Last decoder layer output
        # Add padding to the sides to reach the desired output dimensions
        # Calculate how much padding is needed
        delta_h = 86 - d4.size(2)  # Target height - current height
        delta_w = 64 - d4.size(3)  # Target width - current width
        # Apply padding evenly to the top/bottom and left/right
        pad_top = delta_h // 2
        pad_bottom = delta_h - pad_top
        pad_left = delta_w // 2
        pad_right = delta_w - pad_left
        # Use F.pad to add the required padding
        d4_padded = F.pad(d4, (pad_left, pad_right, pad_top, pad_bottom), mode='constant', value=0)
        # Apply the final output convolution
        out = self.out_conv(d4_padded)
        return out
    
    @staticmethod
    def crop(encoder_layer, decoder_layer):
        """Crop the encoder_layer to the size of the decoder_layer."""
        if encoder_layer.size()[2:] != decoder_layer.size()[2:]:
            encoder_height, encoder_width = encoder_layer.size()[2:]
            decoder_height, decoder_width = decoder_layer.size()[2:]
            # Calculate the difference in height and width
            delta_height = encoder_height - decoder_height
            delta_width = encoder_width - decoder_width
            # Calculate how much to crop from each side
            crop_top = delta_height // 2
            crop_bottom = delta_height - crop_top
            crop_left = delta_width // 2
            crop_right = delta_width - crop_left
            # Crop the encoder layer
            cropped_encoder_layer = encoder_layer[
                :,
                :,
                crop_top:encoder_height - crop_bottom,
                crop_left:encoder_width - crop_right
            ]
            return cropped_encoder_layer
        else:
            # If the sizes match, no cropping is needed
            return encoder_layer

# Instantiate and move the model to GPU
model = UNet(in_channels=3, out_channels=1).cuda()

In [15]:
import torch.optim as optim

criterion = nn.MSELoss()  # For regression tasks
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=30, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=30, shuffle=False)

# Training loop
num_epochs = 50  # Set the number of epochs
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, targets = data
        inputs, targets = inputs.cuda(), targets.cuda()  # Move data to GPU

        optimizer.zero_grad()  # Zero the parameter gradients


        # Forward pass
        outputs = model(inputs)

        # print(f"Output shape: {outputs.shape}")
        # print(f"Target shape: {targets.shape}")

        loss = criterion(outputs, targets)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

    # Validation loop (if needed)
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(val_loader, 0):
            inputs, targets = data
            inputs, targets = inputs.cuda(), targets.cuda()

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

        print(f'Validation Loss after Epoch {epoch + 1}: {val_loss / len(val_loader):.3f}')

print('Finished Training')

Validation Loss after Epoch 1: 3.308
Validation Loss after Epoch 2: 3.095
Validation Loss after Epoch 3: 3.074
Validation Loss after Epoch 4: 2.978
Validation Loss after Epoch 5: 2.877
Validation Loss after Epoch 6: 2.895
Validation Loss after Epoch 7: 2.877
Validation Loss after Epoch 8: 2.871
Validation Loss after Epoch 9: 2.837
Validation Loss after Epoch 10: 2.807
Validation Loss after Epoch 11: 2.780
Validation Loss after Epoch 12: 2.787
Validation Loss after Epoch 13: 2.774
Validation Loss after Epoch 14: 2.795
Validation Loss after Epoch 15: 2.780
Validation Loss after Epoch 16: 2.767
Validation Loss after Epoch 17: 2.799
Validation Loss after Epoch 18: 2.769
Validation Loss after Epoch 19: 2.747
Validation Loss after Epoch 20: 2.754
Validation Loss after Epoch 21: 2.773
Validation Loss after Epoch 22: 2.788
Validation Loss after Epoch 23: 2.772
Validation Loss after Epoch 24: 2.735
Validation Loss after Epoch 25: 2.751
Validation Loss after Epoch 26: 2.738
Validation Loss after